In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import numpy as np
import math
# mpl.rcParams['figure.dpi']= 300
# sns.set(rc={'text.usetex' : True})
import sys
sys.path.append("..")
import src.constants as cnst
from typing import Tuple, List

In [ ]:
DATA_PATH = cnst.DATA_PATH
provided_data = cnst.provided_path
FIG_PATH = cnst.FIG_PATH
df = pd.read_csv(DATA_PATH + provided_data)
df.rename(columns={'PercentPopIncomeBelow2xPovertyLevel': 'PctPvty', 'AvgReduxinNighttimeAnnualTemp_Celsius': 'TempRedux', 'Percent_GreenSpace': 'PctGrSpc'}, inplace=True)

In [ ]:
all_col =pd.read_csv(DATA_PATH + 'all_columns.csv')
all_col

### Optimization Algorithm

In [ ]:
x = np.ones(193) #x is an array of utilities
green_pcts = df["PctGrSpc"].to_numpy()
poverty_pcts = df["PctPvty"].to_numpy()
area = all_col.area #array of areas
population = all_col.population #array of populations
funds = 2000

In [ ]:
def geo_mean(x: np.ndarray) -> float:
    """
    Geometric mean with overflow protection
    """
    return np.exp(np.log(x).mean())

In [ ]:
def green_cost(green_percent: float):
    return 500*(1-(green_percent*0.01))

In [ ]:
def green_change(funds: int, prev_green: float, area: int) -> float:
    """
    Calculate change in green percentage given funds
    """
    return ((funds/green_cost(prev_green)) + (prev_green*area)) / area

In [ ]:
green_change(200, .405, .5)

In [ ]:
from sklearn.linear_model import LinearRegression
X = df.PctGrSpc.to_numpy().reshape(-1, 1)
y = df.TempRedux.to_numpy()
X_train, X_test = X[:-20], X[-20:]
y_train, y_test = y[:-20], y[-20:]
reg = LinearRegression().fit(X_train, y_train)
reg.coef_

In [ ]:
def temp_change(curr_green: float, prev_green: float, model: LinearRegression) -> float:
    """
    Calculate temperature change resulting from additional green coverage
    """
    return model.predict([[curr_green]]) - model.predict([[prev_green]])

In [ ]:
temp_change(70, 20, reg)

In [ ]:
def util(population: int, prev_green: float, curr_green: float, poverty_pct: float, model: LinearRegression) -> float:
    """
    Calculate utility for a tract
    """
    return population * np.log(1 + temp_change(curr_green, prev_green, model)) * np.exp(poverty_pct*0.01)

In [ ]:
def opt(x: np.ndarray, green_pct: np.ndarray, ar: np.ndarray, pop: np.ndarray, pvty_pct: np.ndarray, funds: int, injection: float, reg: LinearRegression) -> Tuple[np.ndarray, float, List]:
    """
    Optimize the utility function for geographic tracts
    Returns geometric mean
    """
    util_delta = []
    util_array = x
    while funds > 0:
        total_util = geo_mean(util_array)
        max_util_delta, max_util_idx = 0, -1
        for idx, u in enumerate(util_array):
            temp = util_array.copy()
            new_green = green_change(injection, green_pct[idx], ar[idx])
            if new_green > 95:
                continue
            else:
                temp[idx] += util(pop[idx], green_pct[idx], new_green, pvty_pct[idx], reg)
                curr_util = geo_mean(temp)
                delta = curr_util - total_util
                if delta >= max_util_delta:
                    max_util_delta = delta
                    max_util_idx = idx
        util_delta.append(max_util_delta)
        ng = green_change(injection, green_pct[max_util_idx], ar[max_util_idx])
        util_array[max_util_idx] += util(pop[max_util_idx], green_pct[max_util_idx], ng, pvty_pct[max_util_idx], reg)
        green_pct[max_util_idx] = ng
        funds -= injection
    return util_array, geo_mean(util_array), util_delta

In [ ]:
test = opt(x.copy(), green_pcts.copy(), area.copy(), population.copy(), poverty_pcts.copy(), funds, 10, reg)

In [ ]:
plt.plot(test[2])

In [ ]:
#plt.plot(test[0])
utilities_after_funding = test[0]
plt.scatter(poverty_pcts, utilities_after_funding)
plt.title("Final Utility vs Percent Poverty")
plt.ylabel("Utility")
plt.xlabel("Percent Population")
plt.savefig("../images/final_utility_vs_poverty")
plt.show()

plt.scatter(green_pcts, utilities_after_funding)
plt.title("Final Utility vs Percent Green Coverage")
plt.ylabel("Utility")
plt.xlabel("Percent Green Coverage")
plt.savefig("../images/final_utility_vs_green_coverage")
plt.show()

plt.scatter(area, utilities_after_funding)
plt.title("Final Utility vs Area")
plt.ylabel("Utility")
plt.xlabel("Area")
plt.savefig("../images/final_utility_vs_area")
plt.show()

plt.scatter(population, utilities_after_funding)
plt.title("Final Utility vs Population")
plt.ylabel("Utility")
plt.xlabel("Population")
plt.savefig("../images/final_utility_vs_population")
plt.show()

In [ ]:
g = sns.scatterplot(data=df, x='lat', y='long', hue=test[0], palette="vlag")
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

n = 100

# For each set of style and range settings, plot n random points in the box
# defined by x in [23, 32], y in [0, 100], z in [zlow, zhigh].
for m, zlow, zhigh in [('o', -50, -25), ('^', -30, -5)]:
    xs = df["lat"]
    ys = df["long"]
    zs = test[0]
    ax.scatter(xs, ys, zs, marker=m)

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
hist, xedges, yedges = np.histogram2d(df["long"], df["lat"], bins=50, range=[[-79.5, -78.5], [35.5, 36.5]])

# Construct arrays for the anchor positions of the 16 bars.
xpos, ypos = np.meshgrid(xedges[:-1] + 0.25, yedges[:-1] + 0.25, indexing="ij")
xpos = xpos.ravel()
ypos = ypos.ravel()
zpos = 0

# Construct arrays with the dimensions for the 16 bars.
dx = dy = 0.02 * np.ones_like(zpos)
dz = hist.ravel()

ax.bar3d(xpos, ypos, zpos, dx, dy, dz, zsort='average')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 10), dpi=150)
ax = fig.add_subplot(121, projection='3d')

x = df["lat"]
y = df["long"]

top = test[0]
bottom = np.zeros_like(top)
width = depth = 0.03

ax.bar3d(x, y, bottom, width, depth, top, shade=True, color="lightblue", )
ax.set_title('Geographic Utility Plot after Planting Trees')
ax.set(xlabel = 'Latitude', ylabel = 'Longitude', zlabel='Utility')
ax.set_zlabel("Utility")

for tick in ax.xaxis.get_majorticklabels():  # example for xaxis
    tick.set_fontsize(8) 
for tick in ax.yaxis.get_majorticklabels():  # example for xaxis
    tick.set_fontsize(8)
plt.show()